**Programming Exercise 1**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import linear_model

np.random.seed(0)

In [2]:
class MiniBatchGradientDescentLinearRegressor:
    ''' Implementation of Mini-Batch Gradient Descent '''
    
    def __init__(self, learning_rate=0.01, batch_size=None, epochs=200):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.X_train = None
        self.y_train = None
    
    def set_params(self, learning_rate=None, epochs=None):
        if learning_rate:
            self.learning_rate = learning_rate
        
        if epochs:
            self.epochs = epochs
    
    def fit(self, X_train, y_train):
        if X_train is None:
            raise Exception('Training feature data has not bee provided')
        
        if y_train is None:
            raise Exception('Training prediction data has not been provided')
            
        self.X_train = X_train
        self.y_train = y_train

In [3]:
class SGDLinearRegressor(MiniBatchGradientDescentLinearRegressor):
    
    def __init__(self, learning_rate, epochs):
        super().__init__(learning_rate=learning_rate, batch_size=1, epochs=epochs)

In [4]:
class BGDLinearRegressor(MiniBatchGradientDescentLinearRegressor):
    
    def __init__(self, learning_rate, batch_size, epochs):
        super().__init__(learning_rate=learning_rate, batch_size=None, epochs=epochs)

In [5]:
!curl -o data.xlsx https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 76189  100 76189    0     0  48190      0  0:00:01  0:00:01 --:--:-- 48159


In [6]:
N = 200

df = pd.read_excel('./data.xlsx')

X_train = df.iloc[:N, :8].to_numpy()
y_train = df.iloc[:N, 8:9].to_numpy()